# IMPORT DES LIBRAIRIES

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

# OPTIONS PANDAS 

In [12]:
pd.options.display.float_format = '{:.0f}'.format

# OUVERTURE DE NOTRE DATASET

In [13]:
df = pd.read_csv('../DATA/OUT/dataset_hors_outliers_surfaces_and_piece.csv')
df.head()

C:\Users\jonat\AppData\Local\Temp\ipykernel_3848\197683011.py:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../DATA/OUT/dataset_hors_outliers_surfaces_and_piece.csv')


,NoDisp,DateMutation,ValeurFonciere,NoVoie,BTQ,TypeVoie,CodeVoie,Voie,CP,Commune,...,Year,Month,AvecTerrain,SurfaceBatiCat,SurfaceCarrez,Localisation,ID,PrixTerrainM2,PrixBatiM2,PrixCarrezM2
0,1,2021-04-26,311650,16,NaN,RUE,0270,DE LA GRANGE DIMIERE,33850,LEOGNAN,...,2021,4,False,"(80.0, 90.0]",84,33238000AA,1,0,3710,3707
1,1,2021-04-26,122000,28,NaN,CRS,0840,DE VERDUN,33470,GUJAN MESTRAS,...,2021,4,False,"(0.999, 35.0]",0,33199000BO,2,0,3588,0
2,1,2021-04-23,420612,47,NaN,RUE,0144,JEAN MARIE PELT,33380,MIOS,...,2021,4,True,"(118.0, 144.0]",0,33284000CT,3,693,3235,0
3,1,2021-04-19,167000,1,NaN,RUE,0030,ARMAND DANEY,33470,GUJAN MESTRAS,...,2021,4,False,"(47.0, 60.0]",49,33199000BO,4,0,3408,3398
4,1,2021-05-05,450000,11,NaN,ALL,1180,FRERES DUPUY CAZAUX,33260,LA TESTE-DE-BUCH,...,2021,5,True,"(100.0, 118.0]",0,33529000CT,5,658,4286,0


# SUPPRESSION DES FEATURES NON ESSENTIELLES

In [14]:
# Préparation de df_model en excluant les colonnes spécifiées dans votre code initial
columns_to_drop = ['NoDisp', 'NoVoie', 'BTQ', 'TypeVoie', 'CodeVoie', 'Voie', 'Commune',
                   'CodeDepartement', 'CodeCommune', 'PrefixeSection', 'Section', 'NoPlan',
                   'PremierLot', 'SurfaceCarrez1erLot', 'DeuxiemeLot', 'SurfaceCarrez2emeLot',
                   'TroisiemeLot', 'SurfaceCarrez3emeLot', 'QuatriemeLot', 'SurfaceCarrez4emeLot',
                   'CinquiemeLot', 'SurfaceCarrez5emeLot','NombreLots', 'NatureCulture', 'NatureCultureSpeciale',
                   'slice', 'quarter', 'Month', 'DateMutation', 'SurfaceBatiCat', 'ID',
                   'PrixTerrainM2', 'PrixBatiM2', 'PrixCarrezM2', 'AvecTerrain', 'Localisation']

# Créer df_model
df_model = df.drop(columns=columns_to_drop)

# Vérifier s'il reste des colonnes non numériques qui doivent être exclues ou traitées séparément
# df_model.select_dtypes(exclude=['number']).columns.tolist()

df_model = df_model.dropna(subset=['ValeurFonciere', 'SurfaceReelleBati', 'CP'])
df_model.isna().sum()

ValeurFonciere             0
CP                         0
CodeTypeLocal              0
SurfaceReelleBati          0
NombrePiecesPrincipales    0
SurfaceTerrain             0
Year                       0
SurfaceCarrez              0
dtype: int64

In [15]:
df_model.shape

(147509, 8)

In [16]:
df_model

,ValeurFonciere,CP,CodeTypeLocal,SurfaceReelleBati,NombrePiecesPrincipales,SurfaceTerrain,Year,SurfaceCarrez
0,311650,33850,1,84,4,0,2021,84
1,122000,33470,1,34,2,0,2021,0
2,420612,33380,1,130,4,607,2021,0
3,167000,33470,2,49,2,0,2021,49
4,450000,33260,1,105,4,684,2021,0
...,...,...,...,...,...,...,...,...
147882,234150,33990,1,103,6,880,2020,0
147883,159300,33340,1,103,5,162,2020,0
147884,159300,33340,1,103,5,500,2020,0
147885,283925,33780,1,132,6,601,2020,0


# TRANSFORMATION DE LOCALISATION EN NUMERIQUE (ONE HOT ENCODER)

In [18]:
# Assurez-vous qu'il n'y a pas de valeurs NaN dans 'CP' avant le One-Hot Encoding
df_model = df_model.dropna(subset=['CP'])

# Réinitialiser l'index pour s'assurer que l'alignement est conservé après la suppression des NaN
df_model.reset_index(drop=True, inplace=True)

# Préparer la colonne Localisation pour le One-Hot Encoding
localisation = df_model[['CP']]

# Instancier le OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
localisation_encoded = encoder.fit_transform(localisation)

# Convertir les résultats encodés en DataFrame
localisation_encoded_df = pd.DataFrame(localisation_encoded,
                                       columns=encoder.get_feature_names_out(['CP']))

# Réinitialiser l'index sur le DataFrame encodé pour garantir l'alignement lors de la fusion
localisation_encoded_df.reset_index(drop=True, inplace=True)

# Fusionner le DataFrame encodé avec le df original en s'assurant que les indices sont alignés
df_model = pd.concat([df_model, localisation_encoded_df], axis=1)

# Supprimer la colonne 'CP' originale
df_model = df_model.drop(columns=['CP'])

In [19]:
df_model

,ValeurFonciere,CodeTypeLocal,SurfaceReelleBati,NombrePiecesPrincipales,SurfaceTerrain,Year,SurfaceCarrez,CP_33000,CP_33100,CP_33110,...,CP_33870,CP_33880,CP_33890,CP_33910,CP_33920,CP_33930,CP_33950,CP_33970,CP_33980,CP_33990
0,311650,1,84,4,0,2021,84,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,122000,1,34,2,0,2021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,420612,1,130,4,607,2021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,167000,2,49,2,0,2021,49,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,450000,1,105,4,684,2021,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147504,234150,1,103,6,880,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147505,159300,1,103,5,162,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147506,159300,1,103,5,500,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147507,283925,1,132,6,601,2020,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# SEPARATION DU JEU DE DONNES

In [6]:
# Séparation du jeu de données en X (caractéristiques) et y (cible)
X, y = df_model.drop(columns=["ValeurFonciere"]), df_model["ValeurFonciere"]

# Séparation en ensembles de train, de test, et de validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.10, random_state=42)

In [7]:
# Instancier le RobustScaler
scaler = RobustScaler()

# Sélectionner les colonnes numériques dans X_train pour l'application du RobustScaler
num_cols = X_train.select_dtypes(include=['number']).columns

# Appliquer le RobustScaler uniquement sur les colonnes numériques de l'ensemble d'entraînement
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Utiliser le scaler ajusté pour transformer les ensembles de test et de validation
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])

print(f'X_train : {X_train.shape[0]}, X_test : {X_test.shape[0]}, X_val : {X_val.shape[0]}')

X_train : 98831, X_test : 43810, X_val : 4868


In [8]:
# Étape 1: Instancier le modèle
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [9]:
# Étape 2: Entraîner le modèle
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [10]:
# Étape 3: Faire des prédictions sur l'ensemble de test
y_pred = rf.predict(X_test)

In [11]:
# Étape 4: Évaluer les performances
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {round(r2*100)} % du prix est expliquer par nos features actuelles")

Mean Squared Error: 9952351318.987082
Mean Absolute Error: 62638.34469012907
R-squared: 69 % du prix est expliquer par nos features actuelles
